<a href="https://colab.research.google.com/github/lenishu/IPA_using_Densenet/blob/main/SLP_weight_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.utils.prune as prune
import numpy as np

# Define a simple fully connected model (SLP) with pruning
class PrunedMLP(nn.Module):
    def __init__(self, prune_percentage=50.0, Prune_Layers='ALL'):
        super(PrunedMLP, self).__init__()
        # Flatten the 28x28 input images (MNIST)
        self.flatten = nn.Flatten()
        # Define a single fully connected layer (input to output)
        self.fc1 = nn.Linear(28 * 28, 10, bias=False)  # Input to output (10 classes for MNIST)
        self.relu = nn.ReLU()

        # Prune the model based on the specified layers
        self.prune_model(prune_percentage, Prune_Layers)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        return x

    def prune_model(self, prune_percentage, Prune_Layers):
        for name, module in self.named_modules():
            if Prune_Layers == 'ALL' and isinstance(module, nn.Linear):
                prune.random_unstructured(module, name='weight', amount=prune_percentage)

def analyze_layer_matrices(model, batch_num, analysis_dir):
    """
    Analyze and save matrix information for each layer
    """
    os.makedirs(analysis_dir, exist_ok=True)

    analysis_file = os.path.join(analysis_dir, f"matrix_analysis_batch_{batch_num}.txt")

    with open(analysis_file, 'w') as f:
        f.write(f"=== MLP MATRIX ANALYSIS - BATCH {batch_num} ===\n\n")

        for name, module in model.named_modules():
            if isinstance(module, nn.Linear):
                # Get the weight matrix (handling pruned weights)
                if hasattr(module, 'weight_mask'):
                    # If pruned, get the actual weights (masked)
                    # Ensure mask is on the same device as weights
                    mask = module.weight_mask.to(module.weight.device)
                    weights = module.weight * mask
                    f.write(f"Layer: {name} (PRUNED)\n")
                else:
                    weights = module.weight
                    f.write(f"Layer: {name} (NOT PRUNED)\n")

                # Convert to numpy for analysis
                weight_np = weights.detach().cpu().numpy()

                # Matrix shape information
                f.write(f"  Matrix Shape: {weight_np.shape}\n")
                f.write(f"  Total Parameters: {weight_np.size}\n")

                # Value range analysis
                min_val = np.min(weight_np)
                max_val = np.max(weight_np)
                mean_val = np.mean(weight_np)
                std_val = np.std(weight_np)

                f.write(f"  Value Range: [{min_val:.6f}, {max_val:.6f}]\n")
                f.write(f"  Mean: {mean_val:.6f}\n")
                f.write(f"  Std Dev: {std_val:.6f}\n")

                # Count zero values (important for pruned networks)
                zero_count = np.sum(weight_np == 0)
                zero_percentage = (zero_count / weight_np.size) * 100
                f.write(f"  Zero Values: {zero_count}/{weight_np.size} ({zero_percentage:.2f}%)\n")

                # Additional statistics for MLP
                non_zero_weights = weight_np[weight_np != 0]
                if len(non_zero_weights) > 0:
                    f.write(f"  Non-zero Mean: {np.mean(non_zero_weights):.6f}\n")
                    f.write(f"  Non-zero Std: {np.std(non_zero_weights):.6f}\n")
                    f.write(f"  Non-zero Range: [{np.min(non_zero_weights):.6f}, {np.max(non_zero_weights):.6f}]\n")

                f.write("\n")

        f.write("=== MLP LAYER SHAPE SUMMARY ===\n")
        f.write("fc1.weight: [10, 784] = 10 output classes, 784 input pixels (28*28)\n")
        f.write("This is a Single Layer Perceptron (SLP) - direct pixel to class mapping\n\n")

def save_layer_matrices(model, batch_num, matrices_dir):
    """
    Save the actual matrix values to separate files
    """
    os.makedirs(matrices_dir, exist_ok=True)

    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            # Get the weight matrix
            if hasattr(module, 'weight_mask'):
                # Ensure mask is on the same device as weights
                mask = module.weight_mask.to(module.weight.device)
                weights = module.weight * mask
            else:
                weights = module.weight

            weight_np = weights.detach().cpu().numpy()

            # Save matrix to file
            filename = os.path.join(matrices_dir, f"{name}_weights_batch_{batch_num}.npy")
            np.save(filename, weight_np)

            # Also save as text file for readability (flattened)
            txt_filename = os.path.join(matrices_dir, f"{name}_weights_batch_{batch_num}_flattened.txt")
            np.savetxt(txt_filename, weight_np.reshape(-1), fmt='%.8f')

            # Save as structured text file preserving matrix shape
            structured_txt_filename = os.path.join(matrices_dir, f"{name}_weights_batch_{batch_num}_structured.txt")
            with open(structured_txt_filename, 'w') as txt_file:
                txt_file.write(f"# Layer: {name}\n")
                txt_file.write(f"# Shape: {weight_np.shape}\n")
                txt_file.write(f"# Batch: {batch_num}\n")
                txt_file.write(f"# MLP Weight Matrix: 10 output classes × 784 input pixels\n")
                txt_file.write("#" + "="*60 + "\n\n")

                # For MLP, show weights organized by output class
                txt_file.write("Weight Matrix (Output Classes × Input Pixels):\n\n")

                for class_idx in range(weight_np.shape[0]):  # 10 classes
                    txt_file.write(f"Class {class_idx} weights (784 pixel connections):\n")

                    # Show first 20×20 pixels of the 28×28 image for readability
                    pixel_weights = weight_np[class_idx].reshape(28, 28)  # Reshape to image format

                    txt_file.write("  First 20×20 pixels:\n")
                    for row in range(min(20, 28)):
                        txt_file.write("  ")
                        for col in range(min(20, 28)):
                            txt_file.write(f"{pixel_weights[row, col]:8.4f} ")
                        if 28 > 20:
                            txt_file.write("...")
                        txt_file.write("\n")

                    if 28 > 20:
                        txt_file.write("  ...\n")

                    # Statistics for this class
                    class_weights = weight_np[class_idx]
                    zero_count = np.sum(class_weights == 0)
                    txt_file.write(f"  Class {class_idx} stats: min={np.min(class_weights):.4f}, ")
                    txt_file.write(f"max={np.max(class_weights):.4f}, ")
                    txt_file.write(f"zeros={zero_count}/784 ({zero_count/784*100:.1f}%)\n\n")

def train_with_analysis(model, train_loader, criterion, optimizer, fraction_of_epoch, device, analysis_dir):
    """
    Training function focused only on matrix analysis
    """
    # Create directories for analysis
    matrices_dir = os.path.join(analysis_dir, "matrices")

    # ANALYZE INITIAL STATE (batch 0)
    print("Analyzing initial state (batch 0)...")
    analyze_layer_matrices(model, 0, analysis_dir)
    save_layer_matrices(model, 0, matrices_dir)

    model.train()
    batch_number = 1
    total_batches = len(train_loader)
    target_batches = int(fraction_of_epoch * total_batches)

    for epoch in range(100):
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            if batch_number > target_batches:
                print(f"Stopping after processing {batch_number} batches")
                return

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # ANALYZE AFTER FIRST BATCH
            if batch_number == 1:
                print("Analyzing after first batch...")
                analyze_layer_matrices(model, 1, analysis_dir)
                save_layer_matrices(model, 1, matrices_dir)

            # ANALYZE AFTER 100 BATCHES
            if batch_number == 100:
                print("Analyzing after 100 batches...")
                analyze_layer_matrices(model, 100, analysis_dir)
                save_layer_matrices(model, 100, matrices_dir)

            print(f"Completed batch {batch_number}/{target_batches}")
            batch_number += 1

def calculate_epochs_for_batch_size(batch_size):
    """
    Calculate number of epochs based on the batch size.
    """
    if batch_size == 64:
        return 1
    elif batch_size == 256:
        return 1
    elif batch_size == 4000:
        return 10
    elif batch_size == 60000:
        return 100
    else:
        raise ValueError("Unsupported batch size")

def main():
    CUDA = True
    device = torch.device("cuda" if CUDA and torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load MNIST dataset
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

    # For demonstration, let's run one configuration
    prune_percentage = 0.5  # 50% pruning
    Prune_Layers = 'ALL'
    batch_size = 60000

    print(f"Running MLP matrix analysis with {prune_percentage*100}% pruning on {Prune_Layers} layers, batch size {batch_size}")

    # Create model
    model = PrunedMLP(prune_percentage, Prune_Layers).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adadelta(model.parameters())

    # Create data loader (only need training data)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Create output directories
    analysis_dir = f"mlp_matrix_analysis_prune_{prune_percentage}_batch_{batch_size}"
    os.makedirs(analysis_dir, exist_ok=True)

    # Calculate epochs
    epochs = calculate_epochs_for_batch_size(batch_size)

    # Train with matrix analysis only
    train_with_analysis(model, train_loader, criterion, optimizer,
                       fraction_of_epoch=epochs, device=device, analysis_dir=analysis_dir)

    print(f"\nMLP Matrix analysis complete! Check the '{analysis_dir}' directory for results.")
    print("Files created:")
    print("- matrix_analysis_batch_0.txt (initial state)")
    print("- matrix_analysis_batch_1.txt (after first batch)")
    print("- matrix_analysis_batch_100.txt (after 100 batches)")
    print("- matrices/ folder with files:")
    print("  * .npy files (NumPy format for loading back)")
    print("  * _flattened.txt (all weights in single column)")
    print("  * _structured.txt (weights organized by output class)")

if __name__ == "__main__":
    main()

Using device: cpu
Running MLP matrix analysis with 50.0% pruning on ALL layers, batch size 60000
Analyzing initial state (batch 0)...
Analyzing after first batch...
Completed batch 1/100
Completed batch 2/100
Completed batch 3/100
Completed batch 4/100
Completed batch 5/100
Completed batch 6/100
Completed batch 7/100
Completed batch 8/100
Completed batch 9/100
Completed batch 10/100
Completed batch 11/100
Completed batch 12/100
Completed batch 13/100
Completed batch 14/100
Completed batch 15/100
Completed batch 16/100
Completed batch 17/100
Completed batch 18/100
Completed batch 19/100
Completed batch 20/100
Completed batch 21/100
Completed batch 22/100
Completed batch 23/100
Completed batch 24/100
Completed batch 25/100
Completed batch 26/100
Completed batch 27/100
Completed batch 28/100
Completed batch 29/100
Completed batch 30/100
Completed batch 31/100
Completed batch 32/100
Completed batch 33/100
Completed batch 34/100
Completed batch 35/100
Completed batch 36/100
Completed batch 